In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%cd DeepLearning_Financial

## EXTERNAL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import yfinance
from pandas import Series
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, PredefinedSplit
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from IPython.display import display
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import time
import os
import random 
from sklearn.datasets import make_regression
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from torch.nn.modules.loss import MSELoss
import tensorflow as tf
from tensorflow import keras
from skorch.dataset import CVSplit
from skorch import callbacks
import pickle
from sklearn.model_selection import train_test_split
from functools import partial
import skorch
import pywt
from sklearn import preprocessing
import joblib


##INTERNAL
from models import Autoencoder, waveletSmooth, SequenceDouble, SequenceDoubleAtt, SequenceAtt
from utils import prepare_data_lstm, ExampleDataset, save_checkpoint, evaluate_lstm, backtest
from testing import *
from sa_tools import *
from test_models import *



#creazione dataset
indices = ['^GSPC']
feature_sets = ['open', 'ohlcv', 'ext']
start_date="2000-01-01"
end_date="2018-12-31"

datasets=get_datasets(indices = indices, feature_sets = feature_sets, end_date=end_date, start_date=start_date)
tss = TimeSeriesSplit(3)#max_train_size
tss_split = CVSplit(cv=tss, stratified=False, random_state=42)

for index in indices:
    for i, (train, val) in enumerate(tss.split(datasets[index]["target"]), start=1):
        for model_name in get_model_names():
            
            # get the model
            save_name = "{}_{}_{}_{}.pkl".format(model_name, index, datasets[index]["target"][train_dates].index.min().date(), datasets[index]["target"][train_dates].index.max().date())
            model, feature_set = get_model(model_name,  save_name)
                
            #get the dataset
            sa=None
            ld=False
            if(feature_set is 'ext_sa'):
                sa = "sa_{}_{}_{}.pkl".format(index, datasets[index]["target"][train_dates].index.min().date(), datasets[index]["target"][train_dates].index.max().date())
            x, y, x_train, x_val, x_scaler, y_train, y_val, y_scaler = get_dataset_train(datasets=datasets, feature_set=feature_set, train_dates=train_dates, val_dates=val_dates, index=index, sa=sa, load=ld)
            
            
            #prepare dataset for training
            n_days=5

            x_batch = days_group(x, n_days=n_days)
            y_batch = y[n_days:]
            serie = x_batch

            splitted = np.split(serie, [len(train_dates_s)])
            l1, l2 = len(splitted[0]), len(splitted[1])
            train_s = PredefinedSplit(np.concatenate((np.ones(l1)*-1,np.ones(l2))))
            train_split =  CVSplit(cv=train_s, stratified=False, random_state=None)
            train_dates_s, val_dates_s = list(perc.split(serie))[0]
            val_dates_s = val_dates_s[val_dates_s < x_batch.shape[0]]
            
            model.train_split = train_split
            model.fit(x_batch,y_batch)
            
            #TODO score
    

/data/home/dsvm_server_admin/notebooks/fastai/tesi/DeepLearning_Financial
{'^GSPC': {'original':                Open     High      Low    Close        Volume
Date                                                        
2000-01-03  1469.25  1478.00  1438.36  1455.22  9.318000e+08
2000-01-04  1455.22  1455.22  1397.43  1399.42  1.009000e+09
2000-01-05  1399.42  1413.27  1377.68  1402.11  1.085500e+09
2000-01-06  1402.11  1411.90  1392.10  1403.45  1.092300e+09
2000-01-07  1403.45  1441.47  1400.73  1441.47  1.225200e+09
...             ...      ...      ...      ...           ...
2018-12-24  2400.56  2410.34  2351.10  2351.10  2.613930e+09
2018-12-25  2400.56  2410.34  2351.10  2351.10  2.613930e+09
2018-12-26  2363.12  2467.76  2346.58  2467.70  4.233990e+09
2018-12-27  2442.50  2489.10  2397.94  2488.83  4.096610e+09
2018-12-28  2498.77  2520.27  2472.89  2485.74  3.702620e+09

[6935 rows x 5 columns], 'features': {}, 'target': Date
2000-01-03    1455.22
2000-01-04    1399.42
2000-01-0